## Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Tables

### Links: 

[[requests_st_support]]

[[intercoms_dir_asgard]]

[[intercoms_st_partner]]

[[companies_dir_partner]]

[[categories_st_support]]
___

In [4]:
query_text = """--sql
CREATE TABLE db1.t_requests_st_support
    (
        `report_date` Date,
        `ticket_id` Int32,
        `intercom_id` String,
        `intercom_uuid` String,
        `camera_id` String,
        `server_stream_url` String,
        `detailed_category` String,
        `category` String,
        `subcategory` String,
        `maincategory` String,  
        `version_os` String,
        `version_app` String,
        `partner_uuid` String,
        `company_name` String,
        `partner_lk` String,
        `com_d_par.company_name` String,
        `device_type` String
    )
    ENGINE = MergeTree()
    ORDER BY report_date
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [5]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.t_requests_st_support_mv
    REFRESH EVERY 1 DAY OFFSET 5 HOUR 35 MINUTE TO db1.t_requests_st_support AS
WITH t1 AS(
SELECT 
    req_s_sup.`report_date` AS report_date, 
    `ticket_id`, 
    `intercom_id`, 
    int_d_as.`intercom_uuid` AS intercom_uuid,
    `camera_id`,
    `server_stream_url`, 
    replaceAll(trim(arrayJoin(splitByChar(',',`category`))),'.','') AS `category`, 
    `version_os`, 
    `version_app`,
    int_s_par.`partner_uuid` AS partner_uuid,
    `company_name`,
    `partner_lk`,
    `device_type`
FROM `db1`.`requests_st_support_ch` AS req_s_sup
LEFT JOIN db1.intercoms_dir_asgard_ch AS int_d_as ON int_d_as.motherboard_id = req_s_sup.intercom_id
LEFT JOIN db1.intercoms_st_partner_ch AS int_s_par 
    ON  int_s_par.intercom_uuid = int_d_as.`intercom_uuid`
    AND int_s_par.report_date = req_s_sup.`report_date`
LEFT JOIN db1.companies_dir_partner_ch AS com_d_par
    ON int_s_par.partner_uuid = com_d_par.partner_uuid
   	),
--
t2 AS (
SELECT
	report_date,
	ticket_id,
	intercom_id,
	intercom_uuid,
	camera_id,
    server_stream_url,
    category,
    if(length(splitByChar(':',category)) = 2, trim(splitByChar(':',category)[2]),trim(splitByChar(':',category)[1])) AS `detailed_category`,
    version_os, 
    version_app,
    partner_uuid,
    company_name,
    partner_lk,
    device_type
FROM t1
)
--
SELECT
	t2.report_date AS report_date,
	ticket_id,
	intercom_id,
	intercom_uuid,
	camera_id,
    server_stream_url,
    category,
    `detailed_category`,
    if(splitByChar('|',Category)[2] = '','Main',splitByChar('|',Category)[2])  AS `subcategory`, 
    splitByChar('|',Category)[1] AS `maincategory`, 
    version_os, 
    version_app,
    partner_uuid,
    company_name,
    partner_lk,
    device_type
FROM t2
JOIN db1.categories_st_support_ch AS cat_s_sup 
	ON cat_s_sup.report_date = t2.report_date
	AND lowerUTF8(cat_s_sup.Details) = lowerUTF8(t2.detailed_category)
WHERE  lowerUTF8(subcategory) != 'info'
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [10]:
query_text = """--sql
    SELECT
        *
    FROM db1.t_requests_st_support
    WHERE subcategory != 'Info'
    ORDER BY report_date
    limit 10
    """

ch.query_run(query_text)


report_date,ticket_id,intercom_id,intercom_uuid,camera_id,server_stream_url,detailed_category,category,subcategory,maincategory,version_os,version_app,partner_uuid,company_name,partner_lk,com_d_par.company_name,device_type
date,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
2025-07-20,196391,"""2428253563""","""18450065-e73a-4135-a2a0-6f2eab…","""""","""msk-v10.sputnik.systems""","""видео с задержкой/зависает/дол…","""MOB|Video: видео с задержкой/з…","""Video""","""MOB""","""iOS""","""4.34.0(12)""","""4d571e14-cd9e-4906-a3c6-121b0e…","""ООО ""ЩИТ-СЕРВИС"" ПРО""","""140661""","""""","""intercom"""
2025-07-20,196473,"""2108117702""","""7f09354a-4e4e-47cd-b4c0-fc9189…","""""","""msk-v03.sputnik.systems""","""домофон не работает""","""DEV: домофон не работает""","""Main""","""DEV""","""iOS""","""4.34.0(12)""","""0e7236ac-b8bd-4ab0-8634-d165ad…","""Цифрал Сервис УК""","""120345""","""""","""intercom"""
2025-07-20,196390,"""2121125457""","""ad8851d2-d2ee-4226-a992-6c5353…","""""","""v08.sputnik.systems""","""приложение не работает""","""MOB: приложение не работает""","""Main""","""MOB""","""iOS""","""4.34.0(12)""","""de03856f-639e-4f54-a77e-dda44b…","""ООО Бастион ПРО""","""120604""","""""","""intercom"""
2025-07-20,196388,"""2148129908""","""b8b973a8-052e-4df7-9d2e-a88f25…","""""","""msk-v02.sputnik.systems""","""домофон не работает""","""DEV: домофон не работает""","""Main""","""DEV""","""Android""","""6.21.1(417)""","""e7e86c75-fc8b-4710-839d-56ad86…","""Цифрал Тольятти""","""152762""","""""","""intercom"""
2025-07-20,196480,"""2125128261""","""6229b775-0185-4baf-b0bd-ff47a6…","""""","""msk-v02.sputnik.systems""","""домофон не работает""","""DEV: домофон не работает""","""Main""","""DEV""","""iOS""","""4.34.0(12)""","""b1d81e5a-be82-46de-bc2a-e9bda5…","""ИП Леонов В.А. ПРО""","""131793""","""""","""intercom"""
2025-07-20,196495,"""1944104592""","""800dbca3-8eab-425d-b144-da0774…","""""","""msk-v04.sputnik.systems""","""дверь не открывается через при…","""MOB|Access: дверь не открывает…","""Access""","""MOB""","""Android""","""6.21.1(417)""","""e7e86c75-fc8b-4710-839d-56ad86…","""Цифрал Тольятти""","""152762""","""""","""intercom"""
2025-07-20,196484,"""2436159160""","""c8e07b6e-c90e-42b5-8bfe-93db50…","""""","""msk-v11.sputnik.systems""","""видеоархива нет/недоступен/не …","""MOB|Storage: видеоархива нет/н…","""Storage""","""MOB""","""Android""","""6.21.1(417)""","""e5f253f6-ea6a-4736-a0b8-6dc9ef…","""АО ""РС Телеком"" ПРО""","""147012""","""""","""intercom"""
2025-07-20,196490,"""2121120584""","""b009f001-aa96-43ba-b1ca-c79388…","""""","""msk-v05.sputnik.systems""","""видео нет/не работает/недоступ…","""MOB|Video: видео нет/не работа…","""Video""","""MOB""","""Android""","""6.21.1(417)""","""e7e86c75-fc8b-4710-839d-56ad86…","""Цифрал Тольятти""","""152762""","""""","""intercom"""
2025-07-20,196490,"""2121120584""","""b009f001-aa96-43ba-b1ca-c79388…","""""","""msk-v05.sputnik.systems""","""история посещений не найдена""","""MOB|Info: история посещений не…","""Video""","""MOB""","""Android""","""6.21.1(417)""","""e7e86c75-fc8b-4710-839d-56ad86…","""Цифрал Тольятти""","""152762""","""""","""intercom"""


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.t_requests_st_support DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


### drop mv

In [2]:
query_text = """--sql
    DROP TABLE db1.t_requests_st_support_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘


### drop ch

In [3]:
query_text = """--sql
    DROP TABLE db1.t_requests_st_support
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [16]:
query_text = """
SYSTEM REFRESH VIEW db1.t_requests_st_support_mv
"""

ch.query_run(query_text)

elapsed_ns,query_id
i64,str
897986,"""58d516b8-81ce-47ef-ab2c-c0196a…"
